<a href="https://colab.research.google.com/github/Samuel442/api-nasa-asteroids/blob/main/script/nasa_asteriodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚙️ **Instalações e Importações**

Bibliotecas principais:
- `requests` → para acessar a API da NASA  
- `pandas` → para organizar e normalizar os dados  

> Obs: Ambas já vêm pré-instaladas no Colab.

```python
import requests
import pandas as pd


In [1]:
# Configuração e Importações
import pandas as pd
import requests
from datetime import date, timedelta
import json

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


## 🔑 Configuração da API e Período

Define:
- **API Key da NASA**
- **data inicial** e **final** da busca  



In [ ]:
# Configuração da API e Datas
API_KEY = "DEMO_KEY"
URL = "https://api.nasa.gov/neo/rest/v1/feed"

# Definir o período de 7 dias para a consulta
# Exemplo: 7 dias terminando em 2025-12-11
end_date = date(2025, 12, 11)
start_date = end_date - timedelta(days=6) # 7 dias de dados

print(f"Período de busca definido: {start_date} a {end_date}")

Período de busca definido: 2025-12-05 a 2025-12-11


## 📥 **3. Extração de Dados (Requisição)**

Esta célula envia a requisição HTTP para a API NeoWs da NASA,
recebe o JSON com os dados dos asteroides e exibe as chaves principais
retornadas pela API.


In [ ]:
# Extração de Dados (Requests)
params = {
    "start_date": start_date.strftime("%Y-%m-%d"),
    "end_date": end_date.strftime("%Y-%m-%d"),
    "api_key": API_KEY
}

try:
    response = requests.get(URL, params=params)
    response.raise_for_status() # Lança erro para códigos 4xx/5xx

    # Carrega o JSON aninhado
    data = response.json()

    print("✅ Dados extraídos com sucesso!")
    # Mostra o número total de elementos que o JSON contém
    print(f"Total de objetos NEO encontrados: {data.get('element_count', 'N/A')}")

except requests.exceptions.HTTPError as err:
    print(f"Erro na requisição HTTP: {err}")
    print("Verifique se as datas estão corretas e se a API Key não atingiu o limite de uso.")

# O objeto de aninhamento a ser processado:
near_earth_objects = data.get("near_earth_objects", {})

✅ Dados extraídos com sucesso!
Total de objetos NEO encontrados: 95


##  **4. Normalização e concatenação**

# O passo de Transformação que usa pd.json_normalize para nivelar o close_approach_data.

In [ ]:
# Processo de Normalização (Transformação)

# Inicializa a lista que guardará os DataFrames de cada dia
dados_aproximacao_df_list = []

# Iterar sobre as CHAVES DE DATA (Ex: "2025-12-05") e suas LISTAS DE ASTEROIDES
for data_aproximacao, lista_asteroides in near_earth_objects.items():

    if lista_asteroides:

        # Expande a lista 'close_approach_data' em novas linhas (record_path)
        # Mantém dados de identificação do asteroide (meta)
        df_aproximacoes_do_dia = pd.json_normalize(
            lista_asteroides,
            record_path='close_approach_data',
            # Metadados do asteroide repetir em cada linha de aproximação:
            meta=['id', 'name', 'is_potentially_hazardous_asteroid'],
            sep='_' # Nívela sub-objetos como miss_distance
        )

        # Adiciona a data de busca para contexto
        df_aproximacoes_do_dia['data_busca'] = data_aproximacao

        dados_aproximacao_df_list.append(df_aproximacoes_do_dia)

# Concatenar todos os resultados
if dados_aproximacao_df_list:
    df_final_normalizado = pd.concat(dados_aproximacao_df_list, ignore_index=True)

    print("\n✅ Normalização e Concatenação Concluídas!")
    print(f"Total final de registros de aproximação: {len(df_final_normalizado)}")
else:
    print("Nenhum dado de asteroide encontrado para normalização.")


✅ Normalização e Concatenação Concluídas!
Total final de registros de aproximação: 95


## 5. 🔍 Inspeção Rápida e Verificação de Tipos
Verifica a estrutura, o cabeçalho (head) e, mais importante, corrige o tipo de dado para que a análise numérica funcione.

In [ ]:
# Verificação e Análise Inicial

# Converte a coluna de distância para float
df_final_normalizado['miss_distance_kilometers'] = \
    df_final_normalizado['miss_distance_kilometers'].astype(float)

print("--- Colunas do DataFrame Final ---")
print(df_final_normalizado.columns)

print("\n--- Top 5 Registros (Aproximações) ---")
# Mostra colunas chave para verificar se a normalização funcionou
colunas_chave = [
    'id',
    'name',
    'data_busca',
    'close_approach_date',
    'miss_distance_kilometers',
    'is_potentially_hazardous_asteroid'
]
print(df_final_normalizado[colunas_chave].head())

# Exemplo de Insight: Encontrar o asteroide que passou mais perto
indice_minimo = df_final_normalizado['miss_distance_kilometers'].idxmin()
asteroide_mais_proximo = df_final_normalizado.loc[indice_minimo]

print("\n--- Asteroide que Passou Mais Perto ---")
print(f"Nome: {asteroide_mais_proximo['name']}")
print(f"Data de Aproximação: {asteroide_mais_proximo['close_approach_date']}")
print(f"Distância Mínima: {asteroide_mais_proximo['miss_distance_kilometers']:.0f} km")

--- Colunas do DataFrame Final ---
Index(['close_approach_date', 'close_approach_date_full',
       'epoch_date_close_approach', 'orbiting_body',
       'relative_velocity_kilometers_per_second',
       'relative_velocity_kilometers_per_hour',
       'relative_velocity_miles_per_hour', 'miss_distance_astronomical',
       'miss_distance_lunar', 'miss_distance_kilometers',
       'miss_distance_miles', 'id', 'name',
       'is_potentially_hazardous_asteroid', 'data_busca'],
      dtype='object')

--- Top 5 Registros (Aproximações) ---
         id              name  data_busca close_approach_date  \
0   2496818  496818 (1993 RA)  2025-12-05          2025-12-05   
1   3826807       (2018 PN22)  2025-12-05          2025-12-05   
2   3836849        (2018 XM2)  2025-12-05          2025-12-05   
3  54224159        (2021 WA1)  2025-12-05          2025-12-05   
4  54356994        (2023 HU5)  2025-12-05          2025-12-05   

   miss_distance_kilometers is_potentially_hazardous_asteroid  
0    

## 5. 🔍 **Inspeção Rápida Data Frame normalizado**

In [ ]:
# Inspeção Rápida do DataFrame Normalizado

print("--- 1. Informações Essenciais ---")
# Mostra o número de linhas, colunas e o uso de memória.
print(df_final_normalizado.info())

print("\n--- 2. Primeiras 5 Linhas (Cabeçalho) ---")
# Exibe as primeiras 5 linhas, confirmando o conteúdo após a normalização.
print(df_final_normalizado.head())

print("\n--- 3. Verificação de Tipos de Dados (dtypes) ---")
# Mostra o tipo de dado de cada coluna
print(df_final_normalizado.dtypes)

# Para inspecionar apenas colunas de interesse (ex: Distância e Velocidade)
print("\n--- 4. dtypes de Colunas Numéricas Chave ---")
colunas_num = [
    'miss_distance_kilometers',
    'relative_velocity_kilometers_per_hour'
]
# A coluna 'miss_distance_kilometers' deve ser 'float64'
print(df_final_normalizado[colunas_num].dtypes)

--- 1. Informações Essenciais ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   close_approach_date                      95 non-null     object 
 1   close_approach_date_full                 95 non-null     object 
 2   epoch_date_close_approach                95 non-null     int64  
 3   orbiting_body                            95 non-null     object 
 4   relative_velocity_kilometers_per_second  95 non-null     object 
 5   relative_velocity_kilometers_per_hour    95 non-null     object 
 6   relative_velocity_miles_per_hour         95 non-null     object 
 7   miss_distance_astronomical               95 non-null     object 
 8   miss_distance_lunar                      95 non-null     object 
 9   miss_distance_kilometers                 95 non-null     float64
 10  miss_distance_mile

### 7.3. 📤 Carregamento para o Supabase
Esta célula usa o método `.to_sql()` do Pandas para criar a tabela e carregar os dados.


In [ ]:
# Inserção Final de Dados via API REST

# Instalação (garante que a biblioteca está pronta)
!pip install supabase

from supabase import create_client, Client

# -----------------------------------------------------------
# CHAVES (Usando o Host HTTPS e a Anon Key fornecida)
# -----------------------------------------------------------
SUPABASE_URL = "SUPABASE_URL"
SUPABASE_KEY = "SUPABASE_ANON_KEY"
TABLE_NAME_API = "neo_close_approaches_api"
# -----------------------------------------------------------

if not SUPABASE_URL or not SUPABASE_KEY:
    raise EnvironmentError(
        "Variáveis de ambiente SUPABASE_URL e SUPABASE_ANON_KEY não configuradas."
    )

try:
    # Cria o cliente Supabase
    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
    print("Cliente Supabase criado com sucesso.")

    # Converte o DataFrame para o formato JSON (lista de dicionários)
    data_to_insert = df_final_normalizado.to_dict('records')
    print(f"Preparando {len(data_to_insert)} registros para inserção...")

    # Insere os dados usando a API REST
    response = (
        supabase.table(TABLE_NAME_API)
        .insert(data_to_insert)
        .execute()
    )

    # Verifica a resposta da API
    if 'data' in response and len(response['data']) > 0:
        print(f"\n✅ SUCESSO! Dados carregados na tabela '{TABLE_NAME_API}'.")
        print(f"Total de registros inseridos: {len(response['data'])}")
    else:
        # Se não houver dados no 'data', mas não houver exceção, pode ser um problema de resposta da API
        # Mas significa que a inserção foi tentada.
        print("\n⚠️ Inserção tentada. Verifique se os dados apareceram no Table Editor do Supabase.")

except Exception as e:
    print(f"❌ Erro Crítico durante a inserção via API: {e}")

Cliente Supabase criado com sucesso.
Preparando 95 registros para inserção...

⚠️ Inserção tentada. Verifique se os dados apareceram no Table Editor do Supabase.


In [ ]:
# Geração do Arquivo CSV para Upload (criar colunas automaticamente com os dados pra ver se bate com o tosql)

# Definir o nome do arquivo de saída
nome_arquivo_csv = "neo_approximations_final.csv"

# Criar o CSV
df_final_normalizado.to_csv(nome_arquivo_csv, index=False, encoding='utf-8')

print(f"\n✅ Arquivo CSV '{nome_arquivo_csv}' criado com sucesso!")

# Download do arquivo (exclusivo para ambiente Google Colab)
from google.colab import files
files.download(nome_arquivo_csv)


✅ Arquivo CSV 'neo_approximations_final.csv' criado com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>